---
title: "Reconstructing Broken Khipus"
jupyter: python3
format:
    html:
        mainfont: et-book, Gill Sans, Tahoma, Verdana, Lucida Grande, Helvetica, Arial
        monofont: Source Code Pro, Menlo, Monaco, 'Courier New', monospace
        backgroundcolor: "#FFFFF8"
        code-fold: true
        code-line-numbers: true
---
<link rel = "stylesheet" type = "text/css" href = "https://www.khipufieldguide.com/css/tufte_fonts.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Source Code Pro">
<hr/>

## Introduction

This page contains companion expository text and code for the article ***"How Can Data Science Contribute to Understanding the Khipu Code?"*** <i>by Manuel Medrano and Ashok Khosla.</i>. More information is available at the Khipu Fieldmarks [Siblings of KH0468-UR231](https://www.khipufieldguide.com/notebook/appendix/notebooks/06%20-%20Siblings%20of%20KH0468-UR231.html) page. 

## 1. Broken Primary Cords

As seen in page 1 (General Khipu Info) section 2.7, there are 156 (24%) khipus with broken primary cords.

## 2. Pendant Pendant Sums for KHO468 (UR231)

### 2.1 Number of Ascher Summations for KHO468 (UR231)

Are UR231A and UR231B two fragments of the same khipu [KH0468/UR231](https://www.khipufieldguide.com/sketchbook/khipu/UR231_wide.html)? We can splice UR231A and UR231B together after cluster 39, where the break occurs. An exhaustive search of the Ascher sum's reveals a wealth of internal pendant-pendant sums arising from the proposed combination of fragments A and B into a single khipu:

In [2]:
from fieldmark_ascher_pendant_pendant_sum import Fieldmark_Pendant_Pendant_Sum
from fieldmark_ascher_pendant_pendants_color_sum import Fieldmark_Pendant_Pendants_Color_Sum
from fieldmark_ascher_indexed_pendant_sum import Fieldmark_Indexed_Pendant_Sum

pendant_pendant_sums_df = Fieldmark_Pendant_Pendant_Sum().dataframes[1].dataframe
pendant_pendant_color_sums_df = Fieldmark_Pendant_Pendants_Color_Sum().dataframes[1].dataframe
indexed_pendant_sums_df = Fieldmark_Indexed_Pendant_Sum().dataframes[1].dataframe

def num_relations(aKhipuName, relations_df):
    return len(relations_df[relations_df.name == aKhipuName])

def num_crossed_relations(aKhipuName, relations_df, left_khipu_num_clusters=39):
    num_crossings = 0
    khipu_df = relations_df[relations_df.name == aKhipuName]
    relation_reps = []
    
    for index, row in khipu_df.iterrows():
        sum_cluster_index = ku.cluster_index_from_cord_rep(row['cord_position'])
        summand_cluster_indices = [ku.cluster_index_from_cord_rep(cord_rep) for cord_rep in row['sum_string'].split('+')]
        (left_summand_cluster_index, right_summand_cluster_index) = (min(summand_cluster_indices), max(summand_cluster_indices))
        is_right_handed = ((sum_cluster_index < left_khipu_num_clusters) and (right_summand_cluster_index >= left_khipu_num_clusters)) #0 based index not 1 based
        is_left_handed = ((sum_cluster_index >= left_khipu_num_clusters) and (left_summand_cluster_index < left_khipu_num_clusters)) #0 based index not 1 based
        
        if (is_left_handed or is_right_handed): 
            sum_string = f"\tSum Cord [{row['cord_name']}]: {row['cord_position']} SummandString: {row['sum_string']}"
            if sum_string not in relation_reps:
                relation_reps.append(sum_string)
                num_crossings += 1
    return (num_crossings, sorted(set(relation_reps)))

for khipu_name in ['XX231L_XX231R']:
    num_pps = num_relations(khipu_name, pendant_pendant_sums_df)
    num_ppscolor = num_relations(khipu_name, pendant_pendant_color_sums_df)
    num_ipps = num_relations(khipu_name, indexed_pendant_sums_df)
    print(f"| {khipu_name} | {num_pps} | {num_ppscolor} | {num_ipps} |")

| XX231L_XX231R | 186 | 13 | 12 |


[XX231L_XX231R](./images/XX231L_XX231R.jpg) - To see Ascher Summation Diagram, click on Number
    
* Count of Pendant Pendant Sums - 186 [(89 left)](./images/XX231L_XX231R_ppsL.jpg)/[(97 right)](./images/XX231L_XX231R_ppsR.jpg) 
* Count of Pendant Pendant Sums by Color - 13 [(5 left)](./images/XX231L_XX231R_ppscL.jpg)/[(6 right)](./images/XX231L_XX231R_ppscR.jpg)  
* Count of Pendant Pendant Sums by Index - 12 [(5 left)](./images/XX231L_XX231R_ippsL.jpg)/[(7 right)](./images/XX231L_XX231R_ippsR.jpg)

### 2.2 Ascher Summations which CROSS the Splice Boundary

How many of these summation relations for the joined XX231L_XX231R **cross** the splice boundary, indicating the two halves are joined correctly?

In [3]:
for khipu_name in ['XX231L_XX231R']:
    print(f"| {khipu_name} | {num_crossed_relations(khipu_name, pendant_pendant_sums_df)[0]} | {num_crossed_relations(khipu_name, pendant_pendant_color_sums_df)[0]} | {num_crossed_relations(khipu_name, indexed_pendant_sums_df)[0]} |")

| XX231L_XX231R | 49 | 9 | 7 |


| Khipu         | # Crossing Pendant Pendant Sums | # Crossing Pendant Pendant Sums by Color | # Crossing Pendant Pendant Sums by Index | 
|:--------------|-----------------------:|--------------------------------:|--------------------------------:|
| XX231L_XX231R | 49 | 9  | 7 | 

: # Crossing Ascher Sums {tbl-colwidths="[22,26,26,26]"}

## 3. Splicing UR231A With Other Possible Khipus

Alternatively we can splice together khipus similar to UR231A and view results. Thee first restriction for similarity is that khipus must have a similar primary cord Ascher color pattern.

### 3.1. Finding Khipus Similar to UR231A

First let's find khipus whose primary cord is similar to UR231A. What is UR231A's primary cord?

In [4]:
(khipu_dict, all_khipus) = kamayuq.fetch_khipus()

ur231 = khipu_dict['UR231']
main_cord = ur231.primary_cord
primary_cord_colors = [color.full_color for color in main_cord.ascher_cord_colors]
primary_cord_colors

['W:AB:KB']

Previous investigations have shown that **cord-color searching by exact color is very unreliable**.  So let's search for primary cords, that are mottled, and have three colors. Let's list possible candidates:

In [5]:
primary_cord_3mottleds = []
rep_string = ""
for khipu in all_khipus:
    if khipu.name().startswith('XX'): continue #Ignore spliced khipus
    main_cord = khipu.primary_cord
    primary_cord_colors = [color.full_color for color in main_cord.ascher_cord_colors]
    has_3mottled_cord = any([color.count(':')==2 for color in primary_cord_colors])
    if has_3mottled_cord: #'W:AB:KB' in primary_cord_colors:
        primary_cord_3mottleds.append(khipu.name())
        rep_string += f"{khipu.name()}: {primary_cord_colors}, "
print(ku.multiline(rep_string, split_char="], ", continuation_char="],\n"))
print(f"{len(primary_cord_3mottleds)} khipus have 3-color mottled primary cords")

AS011: ['YB:B:GG'], AS026B: ['W:GG:FB'], AS044: ['W:BL:B'], AS079: ['W:MB:PB'],
AS139: ['W:MB:CB', 'W'], AS214: ['W:LB:TG'], UR027: ['-:-MB:AB'],
UR046: ['W:AB:MB'], UR053A: ['RL:AB:GG'], UR060: ['AB:MB:HB'],
UR069: ['AB:GG:MB'], UR093: ['AB:MB:KB'], UR1108: ['W:MB:LB'], UR1119: ['W:B:DB'],
UR1120: ['W:GG:LB'], UR117D: ['AB:W-AB:W-MB'], UR139: ['W:AB:KB'],
UR150: ['AB:GG:KB'], UR180: ['PK:AB:KB'], UR227: ['W:AB:BG'], UR231: ['W:AB:KB'],
UR239: ['CB:W-CB:W-AB'], UR244: ['W:AB:KB'], UR246: ['W:AB:MB'],
UR249: ['AB:BG:KB'], KH0033: ['MB-W:GG:AB'], KH0083: ['W:MB:DB'],
QU006: ['YB:BB:PB'], QU009: ['YB:BB:PB'], 
29 khipus have 3-color mottled primary cords


To narrow our search, we'll add an additional constraint. Note that UR231 has:

* The most color bands of any khipu
* It has all verso cords attachments.
* It has all Z knots
* It's mean cords per cluster size is 12
* It has a mean cord value of 27

Accordingly, let's start the search with khipus that have some of these characteristics in common. So let's search for khipus that are banded, with only verso knots, in a Z direction.

In [6]:
# Read in the Mean Cord Value Fieldmark and its associated dataframe and match dictionary
from fieldmark_color_bands import Fieldmark_Color_Bands
color_Fieldmark = Fieldmark_Color_Bands()
color_band_df = color_Fieldmark.dataframes[0].dataframe
banded_khipus_df = color_band_df[color_band_df.num_color_bands > 0]

import fieldmark_khipu_summary as fks
verso_cord_ratio_df = fks.Fieldmark_Verso_Ratio().dataframes[0].dataframe
verso_only_df = verso_cord_ratio_df[verso_cord_ratio_df.v_ratio > .99]

z_knot_ratio_df = fks.Fieldmark_Percent_Z_Knots().dataframes[0].dataframe
z_knot_only_df = z_knot_ratio_df[z_knot_ratio_df.percent_z_knots > .99]

z_knot_verso_only_color_banded_khipus = set(banded_khipus_df.name.to_list()).intersection(set(verso_only_df.kfg_name.to_list())).intersection(set(z_knot_only_df.kfg_name.to_list()))
print(f"{len(z_knot_verso_only_color_banded_khipus)} khipus have z_knots, only verso_attachments, with color_bands")
print(ku.multiline(z_knot_verso_only_color_banded_khipus, split_char=", "))

42 khipus have z_knots, only verso_attachments, with color_bands
{'UR238', 'UR008', 'XX231L_UR022', 'UR252', 'UR091', 'UR213', 'UR221', 'UR056A', 
 'UR089', 'XX231L_UR246', 'UR235', 'UR059', 'UR243', 'KH0001', 'UR200', 'HP015', 
 'UR271', 'UR262', 'UR222', 'HP021', 'UR151', 'HP009', 'XX231L_XX231R', 'UR022', 
 'UR116A', 'HP001', 'UR272', 'UR234', 'XX231L_UR089', 'UR240', 'XX231L_UR088', 
 'UR228', 'UR231', 'UR246', 'UR217', 'UR201', 'UR056B', 'HP029', 'UR088', 'UR134', 
 'HP033', 'HP051 A'}


In [7]:
ur231_matches = sorted(list(set(z_knot_verso_only_color_banded_khipus).intersection(set(primary_cord_3mottleds))))
print(f"{len(ur231_matches)} khipus have z_knots, only verso_attachments, with color_bands, and 3-mottled primary cords")
print(ur231_matches)

2 khipus have z_knots, only verso_attachments, with color_bands, and 3-mottled primary cords
['UR231', 'UR246']


Only 1 Khipu matches, UR246. After evaluating the combination of UR231A and UR246 versus UR231A and UR231B, let's look at the images of the combined khipus, and their associated Ascher Summation Diagrams.

### 3.2. Ascher Summation Diagrams for XX231L_XX246R

X-Ray views are arranged with cords of a cluster in a column. Each square cell in the column represents a cord, and in the center is the cord’s knotted value, and its Ascher color.

Left handed sums (sums whose sum cords are on the right, and summand cords are on the left) have a big bold rectangle on the upper left of the square (Shown as a pink and a red rectangle in the legend). Right handed sums (sums whose sum cords are on the left, and summand cords are on the right) have a big bold rectangle on the upper right of the square (Shown as an orange rectangle in the legend). Colors of the sum are picked cyclically, based on the cluster that the sum cord is contained in. A line of the same color is drawn between the sum cord and the summand cords. Summand Cords are shown on the bottom of the square. Sometimes a cord is used in multiple sums, and in that case the sums are indicated in a row on the bottom of the square.

If a summand cord has a figure 8 knot, then the summand cord is noted with a figure 8 in the center. If a summand cord is adjacent to a cord with a figure 8 knot, then the summand cord is drawn with emphasis.

The longest path (ie. the sum made of the most sums) is drawn vertically on the edge of the column.

[XX231L_XX231R](./images/XX231L_XX231R.jpg) - To see Ascher Summation Diagram, click on Number
    
* Count of Pendant Pendant Sums - 186 [(89 left)](./images/XX231L_XX231R_ppsL.jpg)/[(97 right)](./images/XX231L_XX231R_ppsR.jpg) 
* Count of Pendant Pendant Sums by Color - 13 [(5 left)](./images/XX231L_XX231R_ppscL.jpg)/[(6 right)](./images/XX231L_XX231R_ppscR.jpg)  
* Count of Pendant Pendant Sums by Index - 12 [(5 left)](./images/XX231L_XX231R_ippsL.jpg)/[(7 right)](./images/XX231L_XX231R_ippsR.jpg)

[XX231L_UR246](./images/XX231L_UR246.jpg) - To see Ascher Summation Diagram, click on Number
    
* Count of Pendant Pendant Sums - 96 [(43 left)](./images/XX231L_UR246_ppsL.jpg)/[(53 right)](./images/XX231L_UR246_ppsR.jpg) 
* Count of Pendant Pendant Sums by Color - 8 [(2 left)](./images/XX231L_UR246_ppscL.jpg)/[(6 right)](./images/XX231L_UR246_ppscR.jpg)  
* Count of Pendant Pendant Sums by Index - 6 [(2 left)](./images/XX231L_UR246_ippsL.jpg)/[(4 right)](./images/XX231L_UR246_ippsR.jpg)

In [8]:
for khipu_name in ['XX231L_XX231R', 'XX231L_UR246']:
    num_pps_crosses = num_crossed_relations(khipu_name, pendant_pendant_sums_df, left_khipu_num_clusters=39)[0]
    num_ppscolor_crosses = num_crossed_relations(khipu_name, pendant_pendant_color_sums_df, left_khipu_num_clusters=39)[0]
    num_ipps_crosses = num_crossed_relations(khipu_name, indexed_pendant_sums_df, left_khipu_num_clusters=39)[0]
    print(f"| {khipu_name} | {num_pps_crosses} | {num_ppscolor_crosses} | {num_ipps_crosses} |")

| XX231L_XX231R | 49 | 9 | 7 |
| XX231L_UR246 | 25 | 4 | 4 |


### 3.3. Ascher Summation Diagrams That Cross the Splice Boundary:

| Khipu         | # Crossing Pendant Pendant Sums | # Crossing Pendant Pendant Sums by Color | # Crossing Pendant Pendant Sums by Index | 
|:--------------|-----------------------:|--------------------------------:|--------------------------------:|
| XX231L_XX231R | 49   |  9  |  7 | 
| XX231L_UR246 | 25 | 4 | 4 | 

: # Crossing Ascher Sums {tbl-colwidths="[22,26,26,26]"}